In [1]:
library("tidyverse")
library("SelectSim")

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/ndata/arvind/envs/R_4/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] SelectSim_0.0.1.3 lubridate_1.9.3   forcats_1.0.0     stringr_1.5.1    
 [5] dplyr_1.1.4       purrr_1.0.2       readr_2.1.5       tidyr_1.3.0      
 [9] tibble_3.2.1      ggplot2_3.4.4     tidyverse_2.0.0  

loaded via a nam

In [3]:
filter_run_data <- function(run_data,gene_order,sample_order){
    final_run_data<-run_data
    final_run_data$M$M$missense<-run_data$M$M$missense[gene_order,sample_order]
    final_run_data$M$M$truncating<-run_data$M$M$truncating[gene_order,sample_order]
    final_run_data$alteration.class<- run_data$alteration.class[gene_order]
    final_run_data$sample.class<-run_data$sample.class[sample_order]
    final_run_data$M$tmb$missense<-run_data$M$tmb$missense[sample_order,]
    final_run_data$M$tmb$truncating<-run_data$M$tmb$truncating[sample_order,]
    return(final_run_data)
}

# LUAD

In [4]:
run_data <- readRDS('/mnt/ndata/arvind/co_mutation_project/data/processed/gams/pan_can_tcga_run_data.rds')

In [5]:
sample_covariates<-run_data$sample.class

In [6]:
luad_samples <- names(sample_covariates[(which(sample_covariates %in% c('LUAD')))])

In [7]:
luad_run_data<- filter_run_data(run_data = run_data,gene_order = names(run_data$alteration.class),sample_order = luad_samples)

In [8]:
str(luad_run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:502] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...
  .. ..$ truncating: num [1:396, 1:502] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	502 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...
  .. .. ..$ mutation: num [1:502] 163 253 270 1328 100 ...
  .. ..$ truncating:'data.frame':	502 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-0

In [9]:
obj <- selectX( M = luad_run_data$M,
            sample.class = luad_run_data$sample.class,
            alteration.class = luad_run_data$alteration.class,
            n.cores = 20,
            min.freq = (0.001*length(luad_run_data$sample.class)),
            n.permut = 1000,
            lambda = 0.3,
            tao = 1,
            save.object = FALSE,
            verbose = TRUE,
            estimate_pairwise = FALSE,
            maxFDR = 0.25)

[1] "#### Creating SelectX object ####"
[1] "Step1-> Parsing and Filtering GAM..."
[1] "Number of features: 396"
[1] "Number of features after filtering: 269"
[1] "-> Alteration Landscape object created"
Time:: 0.017 sec elapsed
[1] "Step2-> Generating Templetate object..."
[1] "-> Templetate object created"
Time:: 0.083 sec elapsed
[1] "Step3-> Generating penalty matrix..."
[1] "-> Penalty Matrix created"
Time:: 0.006 sec elapsed
[1] "Step4-> Generating null model..."
[1] "-> Removing the outliers matrix from null model..."
[1] " Removed null-matrix: 105"
[1] " Updated the null-model and nSim variables..."
Time:: 15.762 sec elapsed
[1] "-> Null model generated"
[1] "### SelectX object created ###"
[1] "#### Computing EDs on the dataset ####"
Time:: 18.968 sec elapsed
[1] "#### EDs computed ####"
Total time taken:: 34.836 sec elapsed


In [10]:
obj$result %>% filter(nFDR2<=0.25)  %>% count()

n
<int>
57


## Sampling Run

In [11]:
sample_list1 <- readRDS('/mnt/ptemp/arvind/tool_comaprision/data/sampling_list_luad.rds')
sample_list2 <- readRDS('/mnt/ptemp/arvind/tool_comaprision/data/sampling_list_luad_2.rds')

In [12]:
str(c(sample_list1[11:15],sample_list2[11:15]))

List of 10
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-78-7539-01" "TCGA-99-8033-01" "TCGA-73-4670-01" "TCGA-75-5146-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-75-6214-01" "TCGA-64-5775-01" "TCGA-97-A4M3-01" "TCGA-93-A4JO-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-64-1679-01" "TCGA-64-1676-01" "TCGA-55-8505-01" "TCGA-97-8547-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-55-8204-01" "TCGA-55-8094-01" "TCGA-49-AARE-01" "TCGA-44-A47G-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-05-4422-01" "TCGA-49-AAR9-01" "TCGA-05-4417-01" "TCGA-55-6984-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-91-6848-01" "TCGA-55-6971-01" "TCGA-S2-AA1A-01" "TCGA-86-

In [13]:
sample_list<-c(sample_list1[11:15],sample_list2[11:15])

In [14]:
str(sample_list)

List of 10
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-78-7539-01" "TCGA-99-8033-01" "TCGA-73-4670-01" "TCGA-75-5146-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-75-6214-01" "TCGA-64-5775-01" "TCGA-97-A4M3-01" "TCGA-93-A4JO-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-64-1679-01" "TCGA-64-1676-01" "TCGA-55-8505-01" "TCGA-97-8547-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-55-8204-01" "TCGA-55-8094-01" "TCGA-49-AARE-01" "TCGA-44-A47G-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-05-4422-01" "TCGA-49-AAR9-01" "TCGA-05-4417-01" "TCGA-55-6984-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-91-6848-01" "TCGA-55-6971-01" "TCGA-S2-AA1A-01" "TCGA-86-

In [ ]:
result_vec<-list()
k=1
for(i in c(1:10)){
    load(file='/mnt/ptemp/arvind/select_x_arvind/arvind_tcga_oncko_kb_all_samples_missesne_gam.RData')
    load(file='/mnt/ptemp/arvind/select_x_arvind/arvind_tcga_oncko_kb_all_samples_truncating_gam.RData')
    load('/mnt/ptemp/arvind/giovanni_share/data/gam/tcga_mut_only_marco.RData')
    global_sample_covariates <-gam_collection$sample.class
    gene_to_take <- colnames(missesne_data$gam)
    order <- names(sample_list[[i]])
    data <-list('M'=list('missense'=t(missesne_data$gam[order,gene_to_take]),
                                 'truncating'=t(truncating_data$gam[rownames(missesne_data$gam[order,]),gene_to_take])),
                        'tmb'=list('missense'=missesne_data$tmb[order,],
                                   'truncating'=truncating_data$tmb[order,]))

    alteration_covariates <- rep('MUT',ncol(missesne_data$gam[order,gene_to_take]))
    names(alteration_covariates)<-colnames(missesne_data$gam[order,gene_to_take])
    sample_covariates<-global_sample_covariates[order]
    run_data <- list('M'=data,'sample.class' = sample_covariates,'alteration.class' = alteration_covariates)
    
    obj <- selectX(M = run_data$M,
          sample.class = run_data$sample.class,
          alteration.class =run_data$alteration.class,
          n.cores = 30,
          min.freq=1,
          bootstrap=FALSE,
          n.permut = 1000,
          version=3,
          t_version=2,
          lambda=0.3)
    result <- selectX_table(obj,
                        maxFDR=0.25,
                        n.cores = 1,
                        save.object=FALSE)
    result_vec[[i]]<-result
    saveRDS(result,file=paste('/mnt/ptemp/arvind/tool_comaprision/results/selectX/sampling_run_selectX_v4_result_80_',k,'.rds',sep=""))    
    rm(obj)
    rm(result)
    k=k+1

}